## Regularization:

Seeks to solve a few common issues by:
1. Minimizing the model complexity.
2. Penalizing the loss function.
3. Reducing model overfitting (add more bias to reduce model variance)

### Three main types of regularization:
1. L1 Regularization:
    - LASSO Regression
2. L2 Regularization:
    - Ridge Regression
3. Combining L1 and L2:
    - Elastic Net


## L1 Regularization:
- L1 Regularization adds a penalty equal to the absolute value of the magnitude of coefficients. 
    - Limits the size of the coefficients.
    - Can yield sparse models where some coefficients can become zero.

## L2 Regularization:
- L2 regularization adds a penalty equal to the square of the magnitude of the coefficients.
    - All coefficients are shrunk by the same factor.
    - Does not necessarily eliminate coefficients.
    

## Elastic net:
- Elastic net combines L1 and L2 with the addition of an alpha parameter determining the ratio between them.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
df = pd.read_csv('/home/legacy/anaconda3/Data Science Master Class/UNZIP_FOR_NOTEBOOKS_FINAL/08-Linear-Regression-Models/Advertising.csv')

In [3]:
df.head()


,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [4]:
X = df.drop('sales', axis=1)

In [5]:
y = df['sales']

In [6]:
from sklearn.preprocessing import PolynomialFeatures

In [7]:
polynomial_converter = PolynomialFeatures(degree=3, include_bias=False)

In [8]:
poly_features = polynomial_converter.fit_transform(X)

In [9]:
poly_features.shape

(200, 19)

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.3, random_state=101)

In [12]:
from sklearn.preprocessing import StandardScaler


In [13]:
scaler = StandardScaler()

In [14]:
scaler.fit(X_train)

StandardScaler()

In [15]:
X_train = scaler.transform(X_train)

In [16]:
X_test = scaler.transform(X_test)

## L2 Regularization:

In [17]:
from sklearn.linear_model import Ridge

In [18]:
ridge_model = Ridge(alpha=10)

In [19]:
ridge_model.fit(X_train, y_train) 

Ridge(alpha=10)

In [20]:
test_predictions = ridge_model.predict(X_test)

In [21]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [22]:
MAE = mean_absolute_error(y_test, test_predictions)

In [23]:
MAE

0.5774404204714177

In [24]:
RMSE = np.sqrt(mean_squared_error(y_test, test_predictions))

In [25]:
RMSE

0.894638646131968

 How do we know if alpha=10 was the best choice?

### Use Ridge with Cross-Validation:


In [26]:
from sklearn.linear_model import RidgeCV

In [27]:
ridge_cv_model = RidgeCV(alphas=(0.1, 1.0, 10.0), scoring='neg_mean_absolute_error')

In [28]:
ridge_cv_model.fit(X_train, y_train)

RidgeCV(alphas=array([ 0.1,  1. , 10. ]), scoring='neg_mean_absolute_error')

In [29]:
# Then you can check out the alpha that performed the best:

In [30]:
ridge_cv_model.alpha_

0.1

In [31]:
from sklearn.metrics import SCORERS

In [32]:
SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_wei

In [33]:
test_predictions = ridge_cv_model.predict(X_test)

In [34]:
MAE = mean_absolute_error(y_test, test_predictions)

In [35]:
MAE

0.42737748843313855

In [36]:
RMSE

0.894638646131968

In [37]:
ridge_cv_model.coef_

array([ 5.40769392,  0.5885865 ,  0.40390395, -6.18263924,  4.59607939,
       -1.18789654, -1.15200458,  0.57837796, -0.1261586 ,  2.5569777 ,
       -1.38900471,  0.86059434,  0.72219553, -0.26129256,  0.17870787,
        0.44353612, -0.21362436, -0.04622473, -0.06441449])

## Lasso CV:
Least absolute shrinkage and selection operator


In [38]:
from sklearn.linear_model import Lasso

In [39]:
from sklearn.linear_model import LassoCV

In [40]:
# Lass CV is slighlty different than Ridge CV

In [41]:
lasso_cv_model = LassoCV(eps=0.1, n_alphas=100, cv=5, max_iter=100000)

In [42]:
lasso_cv_model.fit(X_train, y_train) # Careful with the warning, meaning the search didn't converge

LassoCV(cv=5, eps=0.1, max_iter=100000)

In [43]:
lasso_cv_model.alpha_

0.4943070909225828

In [44]:
test_predictions = lasso_cv_model.predict(X_test)

In [45]:
MAE = mean_absolute_error(y_test, test_predictions)

### This model did worse, but it is only taking into consideration a few features out of the 19:


In [46]:
lasso_cv_model.coef_

array([1.002651  , 0.        , 0.        , 0.        , 3.79745279,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ])

In [47]:
from sklearn.linear_model import ElasticNetCV

In [48]:
elastic_model = ElasticNetCV(l1_ratio=[0.1,0.5,0.7,0.9,0.95,0.99,1],eps=0.001,n_alphas=100, max_iter=1000000)

In [49]:
elastic_model.fit(X_train, y_train)

ElasticNetCV(l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1], max_iter=1000000)

In [50]:
elastic_model.l1_ratio

[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1]

In [51]:
# Best ratio:
elastic_model.l1_ratio_

1.0

In [52]:
# Alpha value:
elastic_model.alpha_

0.004943070909225827

In [53]:
test_predictions = elastic_model.predict(X_test)

In [54]:
MAE = mean_absolute_error(y_test, test_predictions)

In [55]:
MAE

0.43350346185900757